In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import cv2

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

master_path = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
master = pd.read_csv(master_path)
master.MRC_ID_DI[master.MRC_ID_DI > 0 ] = 1

In [4]:
master_0 = master[master['MRC_ID_DI'] == 0].sample(frac=1)
master_1 = master[master['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(master_0) if len(master_0) < len(master_1) else len(master_1)

master = pd.concat([master_0.head(sample_size), master_1.head(sample_size)]).sample(frac=1)

In [5]:
from PIL import Image
img_path = os.path.join(os.getcwd(), 'image')

y = []
X = []

for i in range(len(master)):
    path = os.path.join(img_path, str(master.iloc[i, 0]) +'.png')
    label = master.iloc[i, 1]
    img = Image.open(path)
    data = np.asarray(img)
    X.append(data)
    y.append(label)
    
    for ang in range(-20, 20, 5):
        if ang != 0:
            img2 = img.rotate(ang)
            data = np.asarray(img2)
            X.append(data)
            y.append(label)
        
        #img2 = img2.transpose(Image.FLIP_LEFT_RIGHT)
        #data = np.asarray(img2)
        #X.append(data)
        #y.append(label)
        
X = np.array(X)       
y = np.array(y)
y = tf.keras.utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
"""
img_path = os.path.join(os.getcwd(), 'image')

y = []
X_paths = []

for i in range(len(master)):
    X_paths.append( os.path.join(img_path, str(master.iloc[i, 0]) +'.png') )
    y.append(master.iloc[i, 1])

y = np.array(y)
y = tf.keras.utils.to_categorical(y)

X_train_paths, X_test_paths, y_train, y_test = train_test_split(X_paths, y, test_size=0.1, random_state=1)

X_train_paths, X_val_paths, y_train, y_val = train_test_split(X_train_paths, y_train, test_size=0.1, random_state=1)

X_train = []
for file_path in X_train_paths:
    #read image
    img = cv2.imread(file_path)
    X_train.append(img)

X_train = np.array(X_train)

X_val = []
for file_path in X_val_paths:
    #read image
    img = cv2.imread(file_path)
    X_val.append(img)

X_val = np.array(X_val)

X_test = []
for file_path in X_test_paths:
    #read image
    img = cv2.imread(file_path)
    X_test.append(img)

X_test = np.array(X_test)
"""

In [6]:
img_rows, img_cols, img_channel = 224, 224, 3

base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [7]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(tf.keras.layers.Dropout(rate = 0.5))
#add_model.add(tf.keras.layers.Dropout(rate = 0.8))
#add_model.add(tf.keras.layers.Dense(units=8, activation=tf.nn.relu))
add_model.add(tf.keras.layers.Dense(units=2, activation=tf.nn.softmax))

model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0000001),
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
batch_size = 2
epochs = 2000

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255.)#,
        #rotation_range=30, 
        #width_shift_range=0.1,
        #height_shift_range=0.1, 
        #horizontal_flip=True)
train_datagen.fit(X_train)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255.)#,
        #rotation_range=30, 
        #width_shift_range=0.1,
        #height_shift_range=0.1, 
        #horizontal_flip=True)
val_datagen.fit(X_val)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size),
    callbacks=[callback]
)
# callbacks=[tf.keras.callbacks.ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 12467 steps, validate for 1386 steps
Epoch 1/2000
12467/12467 [==============================] - 613s 49ms/step - loss: 0.9083 - accuracy: 0.5132 - val_loss: 0.8386 - val_accuracy: 0.5186oss: 0. - ETA: 39s - loss: 0.9087 - accuracy: 0.513 - ETA: 39s - loss: 0.9088 - accuracy: 0.513 - ETA: 39s - loss: 0.9087 - ETA: 37s - loss: 0.9083 - - ETA: 36s  - ETA: 33s - loss: 0.9079 - ETA: 6s - los - ETA: 4s - loss: 0.9078 - 
Epoch 2/2000
12467/12467 [==============================] - 597s 48ms/step - loss: 0.8430 - accuracy: 0.5488 - val_loss: 0.8515 - val_accuracy: 0.5189
Epoch 3/2000
12467/12467 [==============================] - 598s 48ms/step - loss: 0.7856 - accuracy: 0.5781 - val_loss: 0.8450 - val_accuracy: 0.5200 - - ETA: 7s - loss: 0.7855  - ETA: 4s - loss: 0.7854 - accuracy:  - ETA: 0s - loss: 0.7856 - accu
Epoch 4/2000
12467/12467 [===================

12467/12467 [==============================] - 606s 49ms/step - loss: 0.3360 - accuracy: 0.8625 - val_loss: 0.8257 - val_accuracy: 0.5583 2:12 - ETA: 2:11 - loss: 0.337 - ETA: 2:08 - loss: 0 - ETA: 2:07 - ETA: 2: - ETA: 1:55 - l - ETA - ETA: 1:04 - loss: 0.3371 - accuracy: 0.86 - ETA: 1:04 - loss: 0.3 - ETA: 1:03 - loss: 0.3370 - accura - ETA: 1:02 - loss: - ETA:  - ETA: 51s - loss: 0.3371 - a - ET -
Epoch 18/2000
12467/12467 [==============================] - 603s 48ms/step - loss: 0.3147 - accuracy: 0.8752 - val_loss: 0.8278 - val_accuracy: 0.5630 - ETA: 49s - loss: - ETA: 47s - loss: 0. - ETA:  - ETA:  - ETA: 22s - loss: 0.31 - ETA: 16s - loss: 0
Epoch 19/2000
12467/12467 [==============================] - 601s 48ms/step - loss: 0.2988 - accuracy: 0.8840 - val_loss: 0.8137 - val_accuracy: 0.5586 32s - loss: 0.2998 - accuracy - ETA: 31s - loss: 0.2998 - accuracy:  - ETA: 31s  - ETA: 24s - loss: 0.2998 - accuracy: - ETA: 23s - loss: 0 - ETA: 21s - loss: 0.2996 - a - ETA: 19s - loss: 0

12467/12467 [==============================] - 602s 48ms/step - loss: 0.1384 - accuracy: 0.9700 - val_loss: 0.9314 - val_accuracy: 0.5615 - ETA: 47s - loss - ETA: 17s - loss: 0.1386 - accuracy: - ETA: 16s - loss
Epoch 30/2000
12467/12467 [==============================] - 602s 48ms/step - loss: 0.1246 - accuracy: 0.9740 - val_loss: 0.9348 - val_accuracy: 0.54890.1253 - accura - - ETA: 1:04 - loss: 0.1253 - accuracy - - ETA: 59s - loss: 0.1 - ETA: 57s - loss: 0.1253  - ETA: 36s - loss: 0.1248 - accuracy: 0.97 - ETA: 35s - loss: 0.1249 - accu - ETA: 34s  - ETA:  - ETA: 7s - loss: 0
Epoch 31/2000
12467/12467 [==============================] - 604s 48ms/step - loss: 0.1121 - accuracy: 0.9801 - val_loss: 0.9489 - val_accuracy: 0.5550: 0.1113 - accuracy: 0.98 - ETA: 2:52 - - ETA: 2:44 - loss: 0.1112 - accuracy:  - ETA: 2:44 - loss: 0.1112 - accura - ETA: 2:43 - loss: 0 - ETA: 2:42 - loss: 0.1111 - ac - ETA: 2:31 - loss: 0.1115 - ac - ETA: 2:31 - ETA: 2:25 - - ETA: 2:19 - loss: 0.1 - ETA: 2:1

12467/12467 [==============================] - 603s 48ms/step - loss: 0.0176 - accuracy: 0.9996 - val_loss: 1.2803 - val_accuracy: 0.55073:06 - loss: 0.0197 - accuracy: 1.00 - ETA: 12:46 - loss: 0.0202 - accurac - ETA:  - ETA: 8:49 - loss: 0.0177 - accuracy:  - E - E - ETA: 7:10 - loss: 0.0180 - accu - ETA: 7:10 - ETA - ETA: 7:04 - loss: 0.0182 - ac - ETA: 6:40 - loss: 0.0181 -  - ETA - ETA: 6:35 - loss: 0.0 - E - ETA: 6:00 - loss: 0.0 - ETA: 5:28 - loss: 0.018 - ETA: 5:27 - loss: 0.0180 - accuracy: 0.99 - ETA: 5:27 - loss: 0.0180  - ETA:  - ETA: 5:22 - loss: 0 - ETA: 5:21 - loss: 0.0179 - accuracy:  - ETA: 5:21 - los - ETA: 5: - ETA: 5:18 - loss: 0.017 - ETA: 5:17 - - ETA: 5:15 - loss: 0.0179 - accura - ETA: 5:15 - loss: 0.017 - ETA: 5:14 - l - ETA:  - ETA: 5:09 - loss: 0.017 - ETA: 5: - - ETA - ETA: 5:00 - - - ETA: 4:40 - loss: 0.0177 - ac - ETA: 4:39 - l - ETA: 2:53 - - ETA: 2:50 - loss: - ETA: 2:44 - loss: 0.0177 -  - ETA: 2:43 - loss: 0.0177 - ac - ETA: 2:43 - - ETA: 2:33 - loss: 

12467/12467 [==============================] - 604s 48ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.8496 - val_accuracy: 0.547111 - accuracy:  - ETA: 5:43 - l - ETA: 5:42 - loss: 0.0011 - accu - ETA: 5:41 - los - ETA: 5:40 - los - ETA: 5:28 - loss: 0 - ETA: 5:27 - los - ETA: 5:20 - ETA: 5:18 - loss: 0.001 - ETA: 5:03 - loss: 0.0 - ETA - ETA: 3:44 - loss: 0.0011 - accuracy - ETA: 3:44 - loss: 0.0011 - ac - ETA: 1:00 - loss: 0.0011  - ETA: 59s - loss: 0.00 - ETA: 49s - loss: 0.0011 -  - ETA: 48s - loss: 0.0011 - accuracy: 1. - ETA: 48s - loss: 0.0011 - accuracy: 1.00 - ETA: 47s - loss - ETA: 45s - loss - ETA: 43s - loss:  - ETA: 3 - ETA: 29s - loss: 0.0011 - accurac - ETA: 28s - loss: - ETA: 26s - loss: 0.0011 - accu - ETA:  - ETA
Epoch 59/2000
12467/12467 [==============================] - 609s 49ms/step - loss: 8.3849e-04 - accuracy: 1.0000 - val_loss: 1.9001 - val_accuracy: 0.5402uracy: 1. - ETA: 1:54 - loss: 8 - - ETA: 1:14 - - ETA: 1:12 - loss: 8.4166e - ETA: 1:11 - loss: 

12467/12467 [==============================] - 602s 48ms/step - loss: 4.6834e-05 - accuracy: 1.0000 - val_loss: 2.4966 - val_accuracy: 0.53844.5572e-05 - ac - - ETA: 45s - loss - ETA: 42s - loss: 4 - ETA: 35s - loss: 4.6640e-05  - ETA: 34s - los - ETA: 31s - loss: 4.6604 - 
Epoch 69/2000
12467/12467 [==============================] - 608s 49ms/step - loss: 3.1816e-05 - accuracy: 1.0000 - val_loss: 2.5338 - val_accuracy: 0.5442A: 1:00 - loss: 3.2013e-05 - accuracy: 1.00 - ETA: 1: - ETA: 58s - lo -  - ETA: 50s - loss: 3.2008e-05 - accuracy:  - ETA - ETA
Epoch 70/2000
 7074/12467 [================>.............] - ETA: 4:08 - loss: 2.2574e-05 - accuracy: 1.0000 ETA: 4:52 - loss: 2.2665e-05 -  - ETA: 4:46 - loss: 2.2781e - ETA: 4:41 - loss: 2.2727e - ETA: 4:40 - loss: 2.2759e - ETA: 4:39 - loss: 2.276 - - ETA: 4:36 - loss: 2 - ETA: 4:19 - loss: 2.2626e-05 - accuracy: 1. - ETA: 4:19 - ETA: 4:17 - loss: 2.2608e - ETA: 4:16 - loss: 2.2 - ETA: 4:13 - ETA: 4:09 - loss: 2.2566e-05 - accuracy: WA

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-105d894279b4>", line 27, in <module>
    callbacks=[callback]
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 1306, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\traini

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255.)#,
        #rotation_range=30, 
        #width_shift_range=0.1,
        #height_shift_range=0.1, 
        #horizontal_flip=True)
test_datagen.fit(X_test)

score = model.evaluate(test_datagen.flow(X_test, y_test, batch_size=batch_size), verbose=1)